## This file is a python version of the LPPUB_StatFile_Production.R published by Fannie Mae at this website
https://capitalmarkets.fanniemae.com/credit-risk-transfer/single-family-credit-risk-transfer/fannie-mae-single-family-loan-performance-data

### The output of this file should very closely match the output from the R code by Fannie Mae. The minor differences occur because of how the Fannie Mae R code handles blank spaces. This can be verified by reproducing the statistical summary tables which I did for the year 2007.

#### This code is not commented extensively but one should be able to follow along if they understand the original Fannie Mae R code.

In [1]:
import pandas as pd
import datetime as datetime
#import dask.dataframe as dd
import numpy as np

### Read in the CRT File Layout to get Column types

In [2]:
df_file_desc = pd.read_excel('crt-file-layout-and-glossary.xlsx',sheet_name = 'Combined Glossary' )

df_file_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Field Position                       108 non-null    int64 
 1   Field Name                           108 non-null    object
 2   Description                          108 non-null    object
 3   Date Bound Notes                     34 non-null     object
 4   Respective Disclosure Notes          76 non-null     object
 5   CAS                                  94 non-null     object
 6   CIRT                                 106 non-null    object
 7   Single-Family (SF) Loan Performance  70 non-null     object
 8   Type                                 108 non-null    object
 9   Max Length                           108 non-null    object
dtypes: int64(1), object(9)
memory usage: 8.6+ KB


In [3]:
list_fannie_colnames = ["POOL_ID", "LOAN_ID", "ACT_PERIOD", "CHANNEL", "SELLER", "SERVICER",
                        "MASTER_SERVICER", "ORIG_RATE", "CURR_RATE", "ORIG_UPB", "ISSUANCE_UPB",
                        "CURRENT_UPB", "ORIG_TERM", "ORIG_DATE", "FIRST_PAY", "LOAN_AGE",
                        "REM_MONTHS", "ADJ_REM_MONTHS", "MATR_DT", "OLTV", "OCLTV",
                        "NUM_BO", "DTI", "CSCORE_B", "CSCORE_C", "FIRST_FLAG", "PURPOSE",
                        "PROP", "NO_UNITS", "OCC_STAT", "STATE", "MSA", "ZIP", "MI_PCT",
                        "PRODUCT", "PPMT_FLG", "IO", "FIRST_PAY_IO", "MNTHS_TO_AMTZ_IO",
                        "DLQ_STATUS", "PMT_HISTORY", "MOD_FLAG", "MI_CANCEL_FLAG", "Zero_Bal_Code",
                        "ZB_DTE", "LAST_UPB", "RPRCH_DTE", "CURR_SCHD_PRNCPL", "TOT_SCHD_PRNCPL",
                        "UNSCHD_PRNCPL_CURR", "LAST_PAID_INSTALLMENT_DATE", "FORECLOSURE_DATE",
                        "DISPOSITION_DATE", "FORECLOSURE_COSTS", "PROPERTY_PRESERVATION_AND_REPAIR_COSTS",
                        "ASSET_RECOVERY_COSTS", "MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS",
                        "ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY", "NET_SALES_PROCEEDS",
                        "CREDIT_ENHANCEMENT_PROCEEDS", "REPURCHASES_MAKE_WHOLE_PROCEEDS",
                        "OTHER_FORECLOSURE_PROCEEDS", "NON_INTEREST_BEARING_UPB", "PRINCIPAL_FORGIVENESS_AMOUNT",
                        "ORIGINAL_LIST_START_DATE", "ORIGINAL_LIST_PRICE", "CURRENT_LIST_START_DATE",
                        "CURRENT_LIST_PRICE", "ISSUE_SCOREB", "ISSUE_SCOREC", "CURR_SCOREB",
                        "CURR_SCOREC", "MI_TYPE", "SERV_IND", "CURRENT_PERIOD_MODIFICATION_LOSS_AMOUNT",
                        "CUMULATIVE_MODIFICATION_LOSS_AMOUNT", "CURRENT_PERIOD_CREDIT_EVENT_NET_GAIN_OR_LOSS",
                        "CUMULATIVE_CREDIT_EVENT_NET_GAIN_OR_LOSS", "HOMEREADY_PROGRAM_INDICATOR",
                        "FORECLOSURE_PRINCIPAL_WRITE_OFF_AMOUNT", "RELOCATION_MORTGAGE_INDICATOR",
                        "ZERO_BALANCE_CODE_CHANGE_DATE", "LOAN_HOLDBACK_INDICATOR", "LOAN_HOLDBACK_EFFECTIVE_DATE",
                        "DELINQUENT_ACCRUED_INTEREST", "PROPERTY_INSPECTION_WAIVER_INDICATOR",
                        "HIGH_BALANCE_LOAN_INDICATOR", "ARM_5_YR_INDICATOR", "ARM_PRODUCT_TYPE",
                        "MONTHS_UNTIL_FIRST_PAYMENT_RESET", "MONTHS_BETWEEN_SUBSEQUENT_PAYMENT_RESET",
                        "INTEREST_RATE_CHANGE_DATE", "PAYMENT_CHANGE_DATE", "ARM_INDEX",
                        "ARM_CAP_STRUCTURE", "INITIAL_INTEREST_RATE_CAP", "PERIODIC_INTEREST_RATE_CAP",
                        "LIFETIME_INTEREST_RATE_CAP", "MARGIN", "BALLOON_INDICATOR",
                        "PLAN_NUMBER", "FORBEARANCE_INDICATOR", "HIGH_LOAN_TO_VALUE_HLTV_REFINANCE_OPTION_INDICATOR",
                        "DEAL_NAME", "RE_PROCS_FLAG", "ADR_TYPE", "ADR_COUNT", "ADR_UPB"]

In [4]:
list_variable_types = list(df_file_desc['Type'].values)
dict_column_types = dict(zip(range(len(list_fannie_colnames)),list_variable_types))


In [5]:
list_date_cols = []
list_numeric_cols = []
list_obj_cols=[]
for key,value in dict_column_types.items():
    if value.strip().lower() =='alpha-numeric':
        list_obj_cols.append(key)
        dict_column_types[key]=str
    if value.strip().lower() =='numeric':
        dict_column_types[key]=float
        list_numeric_cols.append(key)
    if value.strip().lower() =='date':
        dict_column_types[key]=str
        list_date_cols.append(key)

In [6]:
list_acquisition_cols =['LOAN_ID', 'ACT_PERIOD', 'CHANNEL', 'SELLER', 'ORIG_RATE', 'ORIG_UPB',
                         'ORIG_TERM', 'ORIG_DATE', 'FIRST_PAY', 'OLTV',
                         'OCLTV', 'NUM_BO', 'DTI', 'CSCORE_B', 'CSCORE_C',
                         'FIRST_FLAG', 'PURPOSE', 'PROP', 'NO_UNITS', 'OCC_STAT',
                         'STATE', 'ZIP', 'MI_PCT', 'PRODUCT', 'MI_TYPE',
                         'RELOCATION_MORTGAGE_INDICATOR']

In [7]:
list_acq_firstperiod_cols=['LOAN_ID','ORIG_CHN','SELLER', 'orig_rt', 'orig_amt','orig_trm', 'orig_date', \
                         'first_pay', 'oltv','ocltv', 'num_bo', 'dti', 'CSCORE_B', 'CSCORE_C','FTHB_FLG', \
                         'purpose', 'PROP_TYP', 'NUM_UNIT', 'occ_stat','state', 'zip_3', 'mi_pct', 'prod_type', \
                         'MI_TYPE','relo_flg']

In [8]:
list_performance_cols = ['LOAN_ID', 'ACT_PERIOD', 'SERVICER', 'CURR_RATE', 'CURRENT_UPB','LOAN_AGE', 'REM_MONTHS',
                          'ADJ_REM_MONTHS', 'MATR_DT', 'MSA','DLQ_STATUS', 'MOD_FLAG', 'Zero_Bal_Code', 'ZB_DTE', 
                          'LAST_PAID_INSTALLMENT_DATE','FORECLOSURE_DATE', 'DISPOSITION_DATE', 'FORECLOSURE_COSTS',
                          'PROPERTY_PRESERVATION_AND_REPAIR_COSTS','ASSET_RECOVERY_COSTS',
                            'MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS', 'ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY', 
                          'NET_SALES_PROCEEDS', 'CREDIT_ENHANCEMENT_PROCEEDS', 'REPURCHASES_MAKE_WHOLE_PROCEEDS',
                          'OTHER_FORECLOSURE_PROCEEDS', 'NON_INTEREST_BEARING_UPB', 'PRINCIPAL_FORGIVENESS_AMOUNT',
                          'repch_flag', 'LAST_UPB']

In [9]:
dict_rename_acq_cols ={'CHANNEL':'ORIG_CHN','ORIG_RATE':'orig_rt',
                    'ORIG_UPB':'orig_amt','ORIG_TERM':'orig_trm',\
                    'ORIG_DATE':'orig_date','FIRST_PAY':'first_pay','OLTV':'oltv','OCLTV':'ocltv',\
                    'NUM_BO':'num_bo','DTI':'dti','FIRST_FLAG':'FTHB_FLG', 'PURPOSE':'purpose', \
                    'PROP':'PROP_TYP','NO_UNITS':'NUM_UNIT', 'OCC_STAT':'occ_stat' , 'STATE':'state',\
                    'ZIP':'zip_3','MI_PCT':'mi_pct', 'PRODUCT':'prod_type',
                    'RELOCATION_MORTGAGE_INDICATOR':'relo_flg'}


In [10]:
dict_rename_perf_cols = {'ACT_PERIOD':'period','SERVICER':'servicer','CURR_RATE':'curr_rte','CURRENT_UPB':'act_upb',\
                    'LOAN_AGE':'loan_age','REM_MONTHS':'rem_mths','ADJ_REM_MONTHS':'adj_rem_months',\
                    'MATR_DT':'maturity_date','MSA':'msa','DLQ_STATUS':'dlq_status', 'MOD_FLAG':'mod_ind',\
                    'Zero_Bal_Code':'z_zb_code','ZB_DTE':'zb_date','LAST_PAID_INSTALLMENT_DATE':'lpi_dte',\
                    'FORECLOSURE_DATE':'fcc_dte', 'DISPOSITION_DATE':'disp_dte', 'FORECLOSURE_COSTS':'FCC_COST',\
                    'PROPERTY_PRESERVATION_AND_REPAIR_COSTS':'PP_COST', 'ASSET_RECOVERY_COSTS':'AR_COST', \
                    'MISCELLANEOUS_HOLDING_EXPENSES_AND_CREDITS':'IE_COST',\
                    'ASSOCIATED_TAXES_FOR_HOLDING_PROPERTY':'TAX_COST', 'NET_SALES_PROCEEDS':'NS_PROCS', \
                    'CREDIT_ENHANCEMENT_PROCEEDS':'CE_PROCS', 'REPURCHASES_MAKE_WHOLE_PROCEEDS':'RMW_PROCS', \
                    'OTHER_FORECLOSURE_PROCEEDS':'O_PROCS', 'NON_INTEREST_BEARING_UPB':'non_int_upb', \
                    'PRINCIPAL_FORGIVENESS_AMOUNT':'prin_forg_upb', 'LAST_UPB':'zb_upb'}

In [11]:
list(dict_rename_acq_cols.keys())

['CHANNEL',
 'ORIG_RATE',
 'ORIG_UPB',
 'ORIG_TERM',
 'ORIG_DATE',
 'FIRST_PAY',
 'OLTV',
 'OCLTV',
 'NUM_BO',
 'DTI',
 'FIRST_FLAG',
 'PURPOSE',
 'PROP',
 'NO_UNITS',
 'OCC_STAT',
 'STATE',
 'ZIP',
 'MI_PCT',
 'PRODUCT',
 'RELOCATION_MORTGAGE_INDICATOR']

In [12]:
input_file_name = '2007Q4'
output_file_name = input_file_name+'_BaseTable7'

In [13]:
start_time = datetime.datetime.now()
#df = dd.read_csv('2010Q1.csv', sep='|', header = None,dtype=object)
#df = pd.read_csv('2010Q3.csv', dtype = dict_column_types)
df = pd.read_csv(input_file_name+'.csv', sep='|',header=None, dtype = dict_column_types)
time_elapsed = datetime.datetime.now()-start_time
df.columns = list_fannie_colnames
time_elapsed

datetime.timedelta(seconds=121, microseconds=405338)

In [14]:
df.iloc[:,list_date_cols]=df.iloc[:,list_date_cols].apply(pd.to_datetime, format='%m%Y')

In [15]:
df['repch_flag']=(df['RPRCH_DTE'].isna()).astype(int)

In [16]:
df_acquisition_data = df.loc[df.groupby('LOAN_ID')['ACT_PERIOD'].idxmin(),list_acquisition_cols]

In [17]:
df_acquisition_data=df_acquisition_data.rename(columns =dict_rename_acq_cols)

In [18]:
df_acquisition_data = df_acquisition_data.replace({'MI_TYPE':{'1':'BPMI','2':'LPMI','3':'IPMI'}})
df_acquisition_data.loc[~df_acquisition_data['MI_TYPE'].isin(['BPMI','LPMI','IPMI']),'MI_TYPE']='None'

In [19]:
df_acquisition_data['ocltv'].fillna(df_acquisition_data['oltv'], inplace = True)
df_acquisition_data=df_acquisition_data.rename(columns = {'first_pay':'FRST_DTE', 'orig_date':'ORIG_DTE'})

### Prepare the Performance variables

In [20]:
df_performance_data = df[list_performance_cols].copy()
df_performance_data = df_performance_data.rename(columns = dict_rename_perf_cols)

In [21]:
df_temp = df_performance_data[['LOAN_ID','period']]
df_last_activity_date = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmax(),:]
df_last_activity_date = df_last_activity_date.rename(columns ={'period':'LAST_ACTIVITY_DATE'})

In [ ]:
df_temp = df_performance_data[['LOAN_ID','period','act_upb']]
df_last_unpd_balance = df_temp.loc[df_temp[df_temp['act_upb']>0].groupby('LOAN_ID')['period'].idxmax(),:]
df_last_unpd_balance= df_last_unpd_balance.rename(columns={'period':'LAST_UPB_DATE','act_upb':'LAST_UPB'})
df_last_unpd_balance = df_last_unpd_balance[['LOAN_ID','LAST_UPB']]

In [ ]:
df_temp = df_performance_data[['LOAN_ID','period','curr_rte']]
dfLastRate = df_temp.loc[df_temp[df_temp['curr_rte']>0].groupby('LOAN_ID')['period'].idxmax(),:]
dfLastRate = dfLastRate.rename(columns = {'curr_rte':'LAST_RT','period':'LAST_RT_DATE'})

In [ ]:
df_temp = df_performance_data[['LOAN_ID','period','z_zb_code']]
df_zb_code = df_temp.loc[df_temp[df_temp['z_zb_code'].isna()==False].groupby('LOAN_ID')['period'].idxmax(),:]
df_zb_code = df_zb_code.rename(columns={'period':'ZB_CODE_DT','z_zb_code' : 'zb_code'})


In [ ]:
df_max_table = pd.merge(df_last_activity_date,df_performance_data, left_on=['LOAN_ID','LAST_ACTIVITY_DATE'], 
                  right_on =['LOAN_ID','period'], how = 'left' )
df_max_table = pd.merge(df_max_table, df_last_unpd_balance, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')

df_max_table = pd.merge(df_max_table, dfLastRate, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')

df_max_table = pd.merge(df_max_table, df_zb_code, left_on = ['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')


In [ ]:
df_temp = df_performance_data[['LOAN_ID','period','servicer']]

df_last_servicer = df_temp.loc[df_temp[df_temp['servicer'].isna()==False].groupby('LOAN_ID')['period'].idxmax(),
                            ['LOAN_ID','servicer']]

df_last_servicer = df_last_servicer.rename(columns={'servicer':'SERVICER'})

In [ ]:
df_performance_data['non_int_upb']=df_performance_data['non_int_upb'].fillna(0)
df_nonint_upb  = df_performance_data[['LOAN_ID','non_int_upb']].\
                groupby('LOAN_ID')['non_int_upb'].max()

df_nonint_upb=df_nonint_upb.to_frame().reset_index()
df_nonint_upb=df_nonint_upb.rename(columns = {'non_int_upb':'NON_INT_UPB'})

In [ ]:
df_base_table2 = pd.merge(df_acquisition_data, df_max_table, left_on =['LOAN_ID'], right_on = ['LOAN_ID'], how = 'left')
df_base_table2 = pd.merge(df_base_table2, df_last_servicer, left_on = ['LOAN_ID'], right_on =['LOAN_ID'], how = 'left')
df_base_table2 = pd.merge(df_base_table2, df_nonint_upb, left_on = ['LOAN_ID'], right_on =['LOAN_ID'], how='left')




### ForeClosure and Delinquency Tables (30, 60, 90, 120, 180 etc)

In [ ]:
df_fcc_table  = df_performance_data[(df_performance_data['lpi_dte'].isna()==False) & 
        (df_performance_data['fcc_dte'].isna()==False) &
        (df_performance_data['disp_dte'].isna()==False)]

In [ ]:
df_fcc_table= df_fcc_table.groupby(['LOAN_ID']).agg({'lpi_dte':'max', 'fcc_dte':'max',
                                     'disp_dte':'max'}).reset_index()


df_fcc_table = df_fcc_table.rename(columns = {'lpi_dte':'LPI_DTE',
                                         'fcc_dte':'FCC_DTE',
                                         'disp_dte':'DISP_DTE'})

In [ ]:
df_base_table3 = pd.merge(df_base_table2, df_fcc_table, left_on =['LOAN_ID'], right_on =['LOAN_ID'], how = 'left')

In [ ]:
df_slim_perf_file = df_performance_data[['LOAN_ID', 'period', 'dlq_status', 'z_zb_code', 'act_upb', 
                                           'zb_upb', 'mod_ind', 'maturity_date', 'rem_mths']]

df_slim_perf_file.loc[:,'dlq_status']=df_slim_perf_file['dlq_status'].replace({'XX':999}).astype(float)

In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['dlq_status']>=1) & \
                                   (df_slim_perf_file['dlq_status']<999) & \
                                    (df_slim_perf_file['z_zb_code'].isna()==True)]

df_f30_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb']]
df_f30_table = df_f30_table.rename(columns= {'period':'F30_DTE','act_upb':'F30_UPB'})


In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['dlq_status']>=2) & \
                                   (df_slim_perf_file['dlq_status']<999) & \
                                    (df_slim_perf_file['z_zb_code'].isna()==True)]

df_f60_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb']]
df_f60_table = df_f60_table.rename(columns= {'period':'F60_DTE','act_upb':'F60_UPB'})



In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['dlq_status']>=3) & \
                                   (df_slim_perf_file['dlq_status']<999) & \
                                    (df_slim_perf_file['z_zb_code'].isna()==True)]

df_f90_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb']]
df_f90_table = df_f90_table.rename(columns= {'period':'F90_DTE','act_upb':'F90_UPB'})




In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['dlq_status']>=4) & \
                                   (df_slim_perf_file['dlq_status']<999) & \
                                    (df_slim_perf_file['z_zb_code'].isna()==True)]

df_f120_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb']]
df_f120_table = df_f120_table.rename(columns= {'period':'F120_DTE','act_upb':'F120_UPB'})





In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['dlq_status']>=6) & \
                                   (df_slim_perf_file['dlq_status']<999) & \
                                    (df_slim_perf_file['z_zb_code'].isna()==True)]

df_f180_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb']]
df_f180_table = df_f180_table.rename(columns= {'period':'F180_DTE','act_upb':'F180_UPB'})






In [ ]:

df_temp = df_slim_perf_file[df_slim_perf_file['z_zb_code'].isin(['02','03','09','15'])| 
                               ((df_slim_perf_file['dlq_status']>=6) &
                                (df_slim_perf_file['dlq_status']<999))]

df_fce_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period','act_upb','zb_upb']]

df_fce_table['FCE_UPB'] = df_fce_table['act_upb']+df_fce_table['zb_upb']

df_fce_table = df_fce_table.rename(columns = {'period':'FCE_DTE'})

df_fce_table = df_fce_table[['LOAN_ID','FCE_DTE','FCE_UPB']]
                                                                
                                                                

### Modifications Table

In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['mod_ind']=='Y') & 
                               (df_slim_perf_file['z_zb_code'].isna()==True)]

df_fmod_date_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period']]
df_fmod_date_table=df_fmod_date_table.rename(columns={'period':'FMOD_DTE'})

In [ ]:
df_temp = df_slim_perf_file[(df_slim_perf_file['mod_ind']=='Y') & 
                               (df_slim_perf_file['z_zb_code'].isna()==True)]
df_fmod_table = pd.merge(df_temp, df_fmod_date_table, left_on =['LOAN_ID'], right_on =['LOAN_ID'], how ='left')
df_fmod_table=df_fmod_table[(df_fmod_table['period']-df_fmod_table['FMOD_DTE'])/np.timedelta64(1, 'M')>=3]


df_fmod_table=df_fmod_table.groupby('LOAN_ID')['act_upb'].max().rename('FMOD_UPB').to_frame().reset_index()

df_fmod_table = pd.merge(df_fmod_table,df_fmod_date_table, left_on = 'LOAN_ID', right_on ='LOAN_ID', how = 'left')
df_fmod_table = pd.merge(df_fmod_table,df_slim_perf_file, left_on =['LOAN_ID','FMOD_DTE'], right_on =
                           ['LOAN_ID', 'period'], how = 'left')
df_fmod_table=df_fmod_table[['LOAN_ID','FMOD_DTE','FMOD_UPB','maturity_date']]

In [ ]:
df_num120_table = pd.merge(df_f120_table, df_acquisition_data, left_on = 'LOAN_ID', right_on ='LOAN_ID', how = 'left')
df_num120_table['z_num_periods_120'] = ((df_num120_table['F120_DTE']-df_num120_table['FRST_DTE'])/np.timedelta64(1,'M')).astype(int)+1
df_num120_table = df_num120_table[['LOAN_ID','z_num_periods_120']]

In [ ]:
df_orig_mat_table = df_slim_perf_file.loc[df_slim_perf_file['maturity_date'].isna()==False, 
                                           ['LOAN_ID','period', 'maturity_date']]
df_orig_mat_table=df_orig_mat_table.loc[df_orig_mat_table.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','maturity_date']]
df_orig_mat_table=df_orig_mat_table.rename(columns={'maturity_date':'orig_maturity_date'})

In [ ]:
df_trm_chng_table = df_slim_perf_file[['LOAN_ID','period','rem_mths']]
df_trm_chng_table.groupby('LOAN_ID')['rem_mths'].diff()

In [ ]:
df_trm_chng_table = df_slim_perf_file.sort_values(['LOAN_ID','period'])
df_trm_chng_table['did_trm_chng'] = df_trm_chng_table.groupby('LOAN_ID')['rem_mths'].diff(1)
df_trm_chng_table = df_trm_chng_table[df_trm_chng_table['did_trm_chng']>=0]
df_trm_chng_table = df_trm_chng_table.loc[df_trm_chng_table.groupby('LOAN_ID')['period'].idxmin(),['LOAN_ID','period']]
df_trm_chng_table = df_trm_chng_table.rename(columns = {'period':'trm_chng_dt'})

In [ ]:
df_mod_trm_table = pd.merge(df_fmod_table,df_orig_mat_table, left_on ='LOAN_ID', right_on ='LOAN_ID', how='left')
df_mod_trm_table = pd.merge(df_mod_trm_table,df_trm_chng_table, left_on ='LOAN_ID', right_on ='LOAN_ID', how='left')


df_mod_trm_table['MODTRM_CHNG']=np.where((df_mod_trm_table['orig_maturity_date']!=df_mod_trm_table['maturity_date'])|
             (df_mod_trm_table['trm_chng_dt'].isna()==False),1,0)
df_mod_trm_table=df_mod_trm_table[['LOAN_ID','MODTRM_CHNG']]


In [ ]:
df_temp = pd.merge(df_slim_perf_file, df_fmod_table, 
                            left_on ='LOAN_ID', right_on ='LOAN_ID', how ='left')
df_temp = df_temp[df_temp['period']<df_temp['FMOD_DTE']]


df_pre_mod_upb_table = df_temp.loc[df_temp.groupby('LOAN_ID')['period'].idxmax(),['LOAN_ID','period','act_upb']]

df_pre_mod_upb_table = df_pre_mod_upb_table.rename(columns ={'period':'pre_mod_period', 'act_upb':'pre_mod_upb'})
df_pre_mod_upb_table = pd.merge(df_pre_mod_upb_table,df_slim_perf_file, left_on=['LOAN_ID','pre_mod_period'],
                          right_on=['LOAN_ID','period'], how='left')


In [ ]:
df_mod_upb_table = pd.merge(df_fmod_table, df_pre_mod_upb_table, left_on='LOAN_ID',right_on='LOAN_ID', how = 'left')
df_mod_upb_table['MODUPB_CHNG'] = np.where(df_mod_upb_table['FMOD_UPB']>=df_mod_upb_table['pre_mod_upb'],1,0)

df_mod_upb_table = df_mod_upb_table[['LOAN_ID','MODUPB_CHNG']]

### The Base Tables as defined in the Fannie R Code, I use BaseTable7 as the final output, which should be similar but not exact to the Fannie Mae R code output.

In [ ]:
df_base_table4 = pd.merge(df_base_table3, df_f30_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_f60_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_f90_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_f120_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_f180_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_fce_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_fmod_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_num120_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_mod_trm_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')
df_base_table4 = pd.merge(df_base_table4, df_mod_upb_table, left_on = 'LOAN_ID', right_on = 'LOAN_ID', how = 'left')


In [ ]:
df_base_table4['F30_UPB']= np.where((df_base_table4['F30_UPB'].isna()==True) & (df_base_table4['F30_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['F30_UPB'])

df_base_table4['F60_UPB'] = np.where((df_base_table4['F60_UPB'].isna()==True) & (df_base_table4['F60_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['F60_UPB'])

df_base_table4['F90_UPB'] = np.where((df_base_table4['F90_UPB'].isna()==True) & (df_base_table4['F90_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['F90_UPB'])

df_base_table4['F120_UPB'] = np.where((df_base_table4['F120_UPB'].isna()==True) & (df_base_table4['F120_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['F120_UPB'])

df_base_table4['F180_UPB'] = np.where((df_base_table4['F180_UPB'].isna()==True) & (df_base_table4['F180_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['F180_UPB'])


df_base_table4['FCE_UPB'] = np.where((df_base_table4['FCE_UPB'].isna()==True) & (df_base_table4['FCE_DTE'].isna()==False), 
         df_base_table4['orig_amt'], df_base_table4['FCE_UPB'])







In [ ]:
df_base_table5 = df_base_table4.copy()

In [ ]:
df_base_table5['LAST_DTE'] = np.where(df_base_table5['disp_dte'].isna()==False, 
                                    df_base_table5['disp_dte'], df_base_table5['LAST_ACTIVITY_DATE'])

df_base_table5['repch_flag'] = np.where(df_base_table5['repch_flag']=='Y', 
                                    1, 0)


df_base_table5['PFG_COST']=df_base_table5['prin_forg_upb']

df_base_table5['MOD_FLAG'] = np.where(df_base_table5['FMOD_DTE'].isna()==False,1, 0)

df_base_table5['MODFG_COST'] = np.where((df_base_table5['mod_ind']=='Y')&(df_base_table5['PFG_COST']>0), 
                                      df_base_table5['PFG_COST'], 0)


df_base_table5['MODTRM_CHNG'] = np.where(df_base_table5['MODTRM_CHNG'].isna()==True,0, df_base_table5['MODTRM_CHNG'])

df_base_table5['MODUPB_CHNG'] = np.where(df_base_table5['MODUPB_CHNG'].isna()==True,0, df_base_table5['MODUPB_CHNG'])

df_base_table5['CSCORE_MN'] = np.where((df_base_table5['CSCORE_C'].isna()==False)&
                                     (df_base_table5['CSCORE_C']<df_base_table5['CSCORE_B']),
                                     df_base_table5['CSCORE_C'],df_base_table5['CSCORE_B'])

df_base_table5['CSCORE_MN'] = np.where(df_base_table5['CSCORE_MN'].isna()==True, 
                                     df_base_table5['CSCORE_B'],df_base_table5['CSCORE_MN'])


df_base_table5['ORIG_VAL'] = df_base_table5['orig_amt']/ (df_base_table5['oltv']/100)

df_base_table5['dlq_status'] = np.where((df_base_table5['dlq_status']=='X')|(df_base_table5['dlq_status']=='XX'),
                                      '999',df_base_table5['dlq_status'])

df_base_table5['z_last_status']=df_base_table5['dlq_status'].astype(float)

df_base_table5['LAST_STAT']='X'
df_base_table5['LAST_STAT'] = df_base_table5['zb_code'].replace({'09':'F','03':'S','02':'T','06':'R','15':'N','16':'L','01':'P'})
                            
df_base_table5.loc[(df_base_table5['z_last_status']<999) & (df_base_table5['z_last_status']>=9),'LAST_STAT']='9'
mask = df_base_table5['z_last_status'].isin([8,7,6,5,4,3,2,1])
df_base_table5.loc[mask, 'LAST_STAT'] = df_base_table5.loc[mask, 'z_last_status'].astype(int).astype(str)

df_base_table5['LAST_STAT']=np.where(df_base_table5['z_last_status']==0,'C',df_base_table5['LAST_STAT'])

df_base_table5['FCC_DTE'] = np.where((df_base_table5['FCC_DTE'].isna()==True) &
                                  (df_base_table5['LAST_STAT'].isin(['F','S','N','T'])),
                                   df_base_table5['zb_date'],df_base_table5['FCC_DTE'])

df_base_table5['COMPLT_FLG'] = np.where(df_base_table5['DISP_DTE'].isna()==False,1,0)
df_base_table5['COMPLT_FLG'] = np.where(~df_base_table5['LAST_STAT'].isin(['F','S','N','T']),
                                      np.nan,df_base_table5['COMPLT_FLG'])


df_base_table5['INT_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) & (df_base_table5['LPI_DTE'].isna()==False),
                                                 df_base_table5['LAST_UPB']-df_base_table5['NON_INT_UPB'],np.nan)


df_base_table5['INT_COST_MONTHS'] = np.where((df_base_table5['COMPLT_FLG']==1) & (df_base_table5['LPI_DTE'].isna()==False),
                                    (df_base_table5['LAST_DTE']-df_base_table5['LPI_DTE'])/np.timedelta64(1, 'M'),
                                           np.nan)

df_base_table5['INT_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) & (df_base_table5['LPI_DTE'].isna()==False),
                                                 (df_base_table5['LAST_UPB']-df_base_table5['NON_INT_UPB'])*\
                                    df_base_table5['INT_COST_MONTHS']*(df_base_table5['LAST_RT']/100-0.0035)/12
                                    ,np.nan)

df_base_table5['INT_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['INT_COST'].isna()==True),
                                    0,df_base_table5['INT_COST'])

df_base_table5=df_base_table5.drop('INT_COST_MONTHS',axis = 1)

df_base_table5['FCC_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['FCC_COST'].isna()==True),
                                    0,df_base_table5['FCC_COST'])
df_base_table5['PP_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['PP_COST'].isna()==True),
                                    0,df_base_table5['PP_COST'])

df_base_table5['AR_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['AR_COST'].isna()==True),
                                    0,df_base_table5['AR_COST'])

df_base_table5['IE_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['IE_COST'].isna()==True),
                                    0,df_base_table5['IE_COST'])

df_base_table5['TAX_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['TAX_COST'].isna()==True),
                                    0,df_base_table5['TAX_COST'])


df_base_table5['PFG_COST'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['PFG_COST'].isna()==True),
                                    0,df_base_table5['PFG_COST'])



df_base_table5['CE_PROCS'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['CE_PROCS'].isna()==True),
                                    0,df_base_table5['CE_PROCS'])


df_base_table5['NS_PROCS'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['NS_PROCS'].isna()==True),
                                    0,df_base_table5['NS_PROCS'])


df_base_table5['RMW_PROCS'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['RMW_PROCS'].isna()==True),
                                    0,df_base_table5['RMW_PROCS'])

df_base_table5['O_PROCS'] = np.where((df_base_table5['COMPLT_FLG']==1) &(df_base_table5['O_PROCS'].isna()==True),
                                    0,df_base_table5['O_PROCS'])

df_base_table5['NET_LOSS'] = np.where((df_base_table5['COMPLT_FLG']==1),df_base_table5['LAST_UPB']+
                                    df_base_table5['FCC_COST']+
                                    df_base_table5['PP_COST']+
                                    df_base_table5['AR_COST']+
                                    df_base_table5['IE_COST']+
                                    df_base_table5['TAX_COST']+
                                    df_base_table5['PFG_COST']+
                                    df_base_table5['INT_COST']-
                                    df_base_table5['NS_PROCS']-
                                    df_base_table5['CE_PROCS']-
                                    df_base_table5['RMW_PROCS']-
                                    df_base_table5['O_PROCS'],np.nan)
                                    
df_base_table5['NET_SEV'] = np.where((df_base_table5['COMPLT_FLG']==1), 
                                   df_base_table5['NET_LOSS']/df_base_table5['LAST_UPB'], np.nan)

 

In [ ]:
df_temp = pd.merge(df_acquisition_data, df_performance_data, left_on = 'LOAN_ID', right_on = 'LOAN_ID', 
                         how = 'left')

df_temp = df_temp.loc[df_temp['mod_ind']=='Y',:]
df_temp['non_int_upb'] = np.where(df_temp['non_int_upb'].isna()==True,0,df_temp['non_int_upb'])
df_temp['modir_cost'] = np.where(df_temp['mod_ind']=='Y',
                                       (df_temp['orig_rt']-df_temp['curr_rte'])\
                                       /1200*df_temp['act_upb'],0)

df_temp['modfb_cost'] = np.where((df_temp['mod_ind']=='Y')&(df_temp['non_int_upb']>0),
                                       (df_temp['curr_rte'])/1200*df_temp['non_int_upb'],0)




df_mod_dir_table = df_temp.groupby('LOAN_ID')[['modir_cost','modfb_cost']].agg('sum').\
                                rename(columns={'modir_cost':'MODIR_COST','modfb_cost':'MODFB_COST'})

df_mod_dir_table['MODTOT_COST'] = df_mod_dir_table['MODIR_COST']+df_mod_dir_table['MODFB_COST']
df_mod_dir_table=df_mod_dir_table.reset_index()



In [ ]:
df_base_table6 = pd.merge(df_base_table5, df_mod_dir_table, left_on = 'LOAN_ID', right_on='LOAN_ID', how ='left')
df_base_table6['non_int_upb'] = np.where((df_base_table6['COMPLT_FLG']==1)&(df_base_table6['non_int_upb'].isna()==True),
                                       0,df_base_table6['non_int_upb']) 


df_base_table6['MOD_COST_MONTHS'] = np.where((df_base_table6['COMPLT_FLG']==1),
                                    (df_base_table5['LAST_DTE']-df_base_table5['zb_date'])/np.timedelta64(1, 'M'),
                                           np.nan)


df_base_table6['MODIR_COST'] = np.where((df_base_table6['COMPLT_FLG']==1),
                                    df_base_table6['MOD_COST_MONTHS']*\
                                (df_base_table6['orig_rt']-df_base_table6['LAST_RT'])/\
                                      1200*df_base_table6['LAST_UPB']+\
                                      df_base_table6['MODIR_COST'],df_base_table6['MODIR_COST'])

df_base_table6['MODFB_COST'] = np.where((df_base_table6['COMPLT_FLG']==1),
                                    df_base_table6['MOD_COST_MONTHS']*\
                                (df_base_table6['LAST_RT'])/\
                                      1200*df_base_table6['non_int_upb']+\
                                      df_base_table6['MODFB_COST'],df_base_table6['MODFB_COST'])






In [ ]:
list_base_table7_cols = ['LOAN_ID','ORIG_CHN','SELLER','orig_rt','orig_amt',
  'orig_trm', 'oltv','ocltv', 'num_bo','dti','CSCORE_B','FTHB_FLG','purpose',
    'PROP_TYP','NUM_UNIT','occ_stat','state','zip_3', 'mi_pct', 'CSCORE_C','relo_flg',
    'MI_TYPE', 'AQSN_DTE','ORIG_DTE','FRST_DTE','LAST_RT','LAST_UPB','msa','FCC_COST',
    'PP_COST','AR_COST','IE_COST','TAX_COST','NS_PROCS','CE_PROCS','RMW_PROCS','O_PROCS',
    'repch_flag','LAST_ACTIVITY_DATE','LPI_DTE','FCC_DTE','DISP_DTE','SERVICER',
    'F30_DTE','F60_DTE','F90_DTE','F120_DTE','F180_DTE','FCE_DTE','F180_UPB','FCE_UPB',
    'F30_UPB','F60_UPB','F90_UPB','MOD_FLAG','FMOD_DTE','FMOD_UPB','MODIR_COST','MODFB_COST',
  'MODFG_COST','MODTRM_CHNG','MODUPB_CHNG','z_num_periods_120','F120_UPB','CSCORE_MN','ORIG_VAL','LAST_DTE',
    'LAST_STAT','COMPLT_FLG','INT_COST', 'PFG_COST', 'NET_LOSS','NET_SEV', 'MODTOT_COST']
list_base_table7_cols.remove('AQSN_DTE')

In [ ]:
df_base_table7 = df_base_table6[list_base_table7_cols]

In [ ]:
df_base_table7.to_csv(output_file_name)